# Plate modal analysis

This modal analysis of a plate works and draws off of a gmsh value to produce the modal analysis. This was confirmed by modeling the NAFEMS plate from the UK in 1990 and confirming the results

# Notes
- Gmsh must be installed and has been added to the docker file
- Only really works in dolfin not dolfinx. this is due to the lack of a good assembly with the boundary conditions needed for the problem

In [1]:
# import stuff


from dolfin import *
import dolfin
comm = dolfin.cpp.MPI.comm_world
rank = comm.Get_rank()
size = comm.Get_size()
print(f"rank: {rank}; size:{size}")
import petsc4py
import numpy as np
import time
import matplotlib.pyplot as plt
import vedo
# %matplotlib notebook
parameters["form_compiler"]["cpp_optimize"] = True
parameters["form_compiler"]["optimize"] = True

rank: 0; size:1


In [2]:


# load in gmsh
mesh = Mesh(comm,"../mesh/extruded_box_panel_200.xml")
# # In[2]:

E, nu = Constant(200e9), Constant(0.3)
rho = Constant(8000)


In [5]:

# Lame coefficient for constitutive relation
mu = E/2./(1+nu)
lmbda = E*nu/(1+nu)/(1-2*nu)

def eps(v):
    return sym(grad(v))

def sigma(v):
    dim = v.geometric_dimension()
    return 2.0*mu*eps(v) + lmbda*tr(eps(v))*Identity(dim)



In [6]:

V = VectorFunctionSpace(mesh, 'Lagrange', degree=1)
u_ = TrialFunction(V)
du = TestFunction(V)

## Notes on Boundary Condition

The simply supported boundary condition (Z =0) is applied to the third subspace of the function space. V.sub(2)

In [7]:

def simply_supported(x,on_boundary):
    return on_boundary and (near(x[0],0.0) | near(x[0],1.0)| near(x[1],0.0)| near(x[1],1.0))



bc = DirichletBC(V.sub(2),Constant((0.)), simply_supported)



## Notes on building system
There are known symmetry issues with building the system with the K matrix. This solves them for uknown reasons

In [8]:


k_form = inner(sigma(du),eps(u_))*dx
l_form = Constant(1.)*u_[0]*dx
K = PETScMatrix(comm)
b = PETScVector(comm)
assemble_system(k_form, l_form,bc, A_tensor=K, b_tensor=b)


(<dolfin.cpp.la.PETScMatrix at 0x7fba89874e08>,
 <dolfin.cpp.la.PETScVector at 0x7fba89874e60>)

In [9]:

m_form = rho*dot(du,u_)*dx
M = PETScMatrix(comm)
assemble(m_form, tensor=M)
bc.zero(M)


In [10]:


eigensolver = SLEPcEigenSolver(K,M)
eigensolver.parameters['problem_type'] = 'gen_hermitian'
eigensolver.parameters['spectral_transform'] = 'shift-and-invert'
eigensolver.parameters['spectral_shift'] = 600.
eigensolver.parameters['maximum_iterations'] = 500000


In [11]:

N_eig = 50   # number of eigenvalues
print("Computing {} first eigenvalues...".format(N_eig))
start = time.perf_counter()
eigensolver.solve(N_eig)
print(f"elapsed time: {time.perf_counter()-start}")
print(f"N Solutions Found: {eigensolver.get_number_converged()}")


Computing 50 first eigenvalues...
elapsed time: 1.9637473999755457
N Solutions Found: 51


In [10]:
eigensolver.parameters['tolerance']

In [12]:

if rank == 0:
    for i in range(N_eig):
        # Extract eigenpair
        r, c, rx, cx = eigensolver.get_eigenpair(i)
        # 3D eigenfrequency
        freq_2D = sqrt(r)/2/pi
        print(f"Solid FEM: {freq_2D:.3f} Hz")



Solid FEM: 0.001 Hz
Solid FEM: 0.001 Hz
Solid FEM: 0.001 Hz
Solid FEM: 125.136 Hz
Solid FEM: 312.596 Hz
Solid FEM: 313.546 Hz
Solid FEM: 497.776 Hz
Solid FEM: 624.845 Hz
Solid FEM: 627.431 Hz
Solid FEM: 807.156 Hz
Solid FEM: 809.040 Hz
Solid FEM: 1058.695 Hz
Solid FEM: 1061.548 Hz
Solid FEM: 1115.200 Hz
Solid FEM: 1239.115 Hz
Solid FEM: 1241.018 Hz
Solid FEM: 1539.154 Hz
Solid FEM: 1544.644 Hz
Solid FEM: 1610.379 Hz
Solid FEM: 1615.635 Hz
Solid FEM: 1787.523 Hz
Solid FEM: 1790.593 Hz
Solid FEM: 1937.026 Hz
Solid FEM: 1964.902 Hz
Solid FEM: 2067.983 Hz
Solid FEM: 2068.164 Hz
Solid FEM: 2086.386 Hz
Solid FEM: 2093.092 Hz
Solid FEM: 2193.173 Hz
Solid FEM: 2277.453 Hz
Solid FEM: 2286.683 Hz
Solid FEM: 2454.592 Hz
Solid FEM: 2457.784 Hz
Solid FEM: 2498.473 Hz
Solid FEM: 2504.319 Hz
Solid FEM: 2509.310 Hz
Solid FEM: 2748.289 Hz
Solid FEM: 2753.883 Hz
Solid FEM: 2882.616 Hz
Solid FEM: 3034.756 Hz
Solid FEM: 3055.350 Hz
Solid FEM: 3069.629 Hz
Solid FEM: 3110.719 Hz
Solid FEM: 3111.420 Hz
Solid

In [13]:
import pyvista as pv
from vedo import Box
import vedo
vedo.embedWindow(False)
pv.start_xvfb()

In [14]:
eigenmodes = []
# box = Box(length=1, width=1, height=1).pos(0.5,0,0).wireframe()
for i in range(N_eig):
    # Extract eigenpair
    r, c, rx, cx = eigensolver.get_eigenpair(i)

    # 3D eigenfrequency
    freq_3D = sqrt(r)/2/pi

   
    # Initialize function and assign eigenvector
    eigenmode = Function(V,name="Eigenvector "+str(i))
    eigenmode.vector()[:] = rx

#     eigenmodes.append(eigenmode)
    vedo.dolfin.plot(eigenmode,offscreen=True)
    vedo.screenshot(filename=f"../data/Candidate_3d_Plate/Candidate_3D_PLATE_MODE_{i}_{freq_3D:.2f}_Hz")

#     file_results.write(eigenmode,0.)
    


 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh...    
 ━━━━━━━━━━━━━━━━━━━━━━                     converting mesh..

In [19]:
eigenmode.vector()[:]

array([ 0.00014492,  0.0001105 ,  0.        , ..., -0.00009351,
       -0.00009237,  0.        ])

In [ ]:
# visualie the data by exporting to file


# file_results = XDMFFile("../data/modal_analysis.xdmf")
# file_results.parameters["flush_output"] = True
# file_results.parameters["functions_share_mesh"] = True
# file_results.write(eigenmode, 0.)


# Visualization
## Notes
- Ne3eds to have vedo installed

In [ ]:
from vedo.dolfin import *
from vedo import Box
import vedo
vedo.embedWindow(False)

In [ ]:
# add a frame box


In [ ]:
# iterate over the modes nadp lot them

for idx,mode in enumerate(eigenmodes):
        
    vedo.dolfin.plot(mode,box,offscreen=True)
    vedo.screenshot(filename=f"../data/NAFEMS_PLATE_MODE_{idx}")
